In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['figure.dpi'] = 500
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import LogNorm
from matplotlib.patches import FancyArrowPatch
from astropy.visualization import SqrtStretch, LinearStretch, LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import pandas as pd
import re
from astropy.io import fits
import os
from scipy.ndimage import map_coordinates
from scipy.stats import binned_statistic
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

from matplotlib.lines import Line2D
os.chdir("/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling")

In [2]:
# Model Opening
model_path = 'Test/SIE+SHEAR'
model_ver = 'SIE_POS_SHEAR'
lens_name = f'"{model_path}"'

if 'POS+FLUX' in model_ver:
    constraint = 'pos_flux'
elif 'POS' in model_ver:
    constraint = 'pos'


# Load the data
with open(model_path + '/' + model_ver + '_optresult' + '.dat', 'r') as file:
    opt_result = file.readlines()

opt_result

['------------------------------------------\n',
 'optimize ndim=7\n',
 'run 1: 128 lens models calculated\n',
 'chi^2 = 1.838984e+02  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' point no 1 : 1.838984e+02 1.838984e+02 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' lens prior : 0.000000e+00\n',
 ' map prior  : 0.000000e+00\n',
 '\n',
 'omega = 0.3000  lambda = 0.7000  weos = -1.0000  hubble = 0.7000\n',
 '\n',
 'lens   sie     0.2613  1.580400e+02  2.080916e+01  2.075766e+01  1.289154e-01 -4.431012e+00  0.000000e+00  0.000000e+00 \n',
 'lens   pert    0.2613  1.000000e+00  2.078000e+01  2.078000e+01  1.233526e-04  1.527151e+02  0.000000e+00  1.854468e-03 \n',
 'point  1.0000  2.081223e+01  2.077895e+01 \n',
 '------------------------------------------\n',
 '------------------------------------------\n',
 'optimize ndim=7\n',
 'run 2: 22 lens models calculated\n',
 'chi^2 = 1.838952e+02  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.00000